# Poster Figs nb for SOSE

## 1. First create datasets for WG variable

In [1]:
from dask.distributed import Client

client = Client("tcp://10.32.16.186:33841")
client

Client Scheduler: tcp://10.32.16.186:33841 Dashboard: /user/0000-0002-8176-9465/proxy/8787/status,Cluster Workers: 30 Cores: 60 Memory: 345.00 GB


In [2]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import intake
import fsspec
import xmitgcm
from xhistogram.xarray import histogram
from xmitgcm.llcreader.llcmodel import faces_dataset_to_latlon
%matplotlib inline

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['geojson', 'postgis', 'shapefile', 'spatialite'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [3]:
xr.set_options(display_style='text')

In [4]:
#WG boundaries (65˚W, 30˚E; 80˚S, 62˚S)
#WG model's boundaries (-77.87497˚S, -62.041634˚S; 295.08334W, 30.083334E)
lower_lat = 0
upper_lat = 95
left_lon =  310
right_lon = 880
XG_left = 309
XG_right = 881
YG_upper = 96

In [5]:
#set the boundaries
lat_range = dict(YC=slice(lower_lat, upper_lat), YG=slice(lower_lat, YG_upper))
lon_range = dict(XC=slice(left_lon, right_lon), XG=slice(left_lon, XG_right))

In [6]:
#set the boundaries
lat_range_noface = dict(YC=slice(lower_lat, upper_lat))
lon_range_noface = dict(XC=slice(left_lon, right_lon))

In [7]:
base_dir = '/home/jovyan/WMT/SOSE-budgets/datasets/'
coords = xr.open_dataset('/home/jovyan/WMT/SOSE-budgets/datasets/coords_wg.nc')
ds_tns = xr.open_dataset('/home/jovyan/WMT/SOSE-budgets/datasets/ds_tns.nc')
ds_slt = xr.open_dataset('/home/jovyan/WMT/SOSE-budgets/datasets/ds_slt_wg.nc')
ds_velocity = xr.open_dataset('/home/jovyan/WMT/SOSE-budgets/datasets/ds_vel.nc')
ds = xr.merge([ds_tns, ds_slt, ds_velocity])
ds

<xarray.Dataset>
Dimensions:                   (XC: 570, XG: 571, YC: 95, YG: 96, Z: 42, Zl: 42, time: 438)
Coordinates:
  * XC                        (XC) float32 295.08334 295.25 ... 29.75 29.916668
  * YC                        (YC) float32 -77.87497 -77.7083 ... -62.2083
  * Z                         (Z) float32 -5.0 -15.5 -27.0 ... -5325.0 -5575.0
  * time                      (time) datetime64[ns] 2005-01-06 ... 2010-12-31
  * XG                        (XG) float32 295.0 295.1667 ... 29.833334 30.0
  * YG                        (YG) float32 -77.9583 -77.79163 ... -62.124966
  * Zl                        (Zl) float32 0.0 -10.0 -21.0 ... -5200.0 -5450.0
Data variables:
    THETA                     (time, Z, YC, XC) float32 ...
    SALT                      (time, Z, YC, XC) float32 ...
    ETAN                      (time, YC, XC) float32 ...
    conv_horiz_adv_flux_SLT   (time, Z, YC, XC) float32 ...
    conv_horiz_diff_flux_SLT  (time, Z, YC, XC) float32 ...
    conv_vert_adv_flux_SLT    (time, Z, YC, XC) float32 ...
    conv_vert_diff_flux_SLT   (time, Z, YC, XC) float32 ...
    lin_fs_correction_SLT     (time, YC, XC) float32 ...
    surface_flux_conv_SLT     (time, YC, XC) float32 ...
    total_tendency_SLT_truth  (time, Z, YC, XC) float64 ...
    UVEL                      (time, Z, YC, XG) float32 ...
    VVEL                      (time, Z, YG, XC) float32 ...
    WVEL                      (time, Zl, YC, XC) float32 ...
    WTHMASS                   (time, Zl, YC, XC) float32 ...
    WSLTMASS                  (time, Zl, YC, XC) float32 ...
    surForcS                  (time, YC, XC) float32 ...
    surForcT                  (time, YC, XC) float32 ...

In [8]:
coords_no_polynya = coords.isel(time=slice(72,  -1))
ds_no_polynya = ds.isel(time=slice(72,  -1))

In [9]:
ds_no_polynya

<xarray.Dataset>
Dimensions:                   (XC: 570, XG: 571, YC: 95, YG: 96, Z: 42, Zl: 42, time: 365)
Coordinates:
  * XC                        (XC) float32 295.08334 295.25 ... 29.75 29.916668
  * YC                        (YC) float32 -77.87497 -77.7083 ... -62.2083
  * Z                         (Z) float32 -5.0 -15.5 -27.0 ... -5325.0 -5575.0
  * time                      (time) datetime64[ns] 2006-01-01 ... 2010-12-26
  * XG                        (XG) float32 295.0 295.1667 ... 29.833334 30.0
  * YG                        (YG) float32 -77.9583 -77.79163 ... -62.124966
  * Zl                        (Zl) float32 0.0 -10.0 -21.0 ... -5200.0 -5450.0
Data variables:
    THETA                     (time, Z, YC, XC) float32 ...
    SALT                      (time, Z, YC, XC) float32 ...
    ETAN                      (time, YC, XC) float32 ...
    conv_horiz_adv_flux_SLT   (time, Z, YC, XC) float32 ...
    conv_horiz_diff_flux_SLT  (time, Z, YC, XC) float32 ...
    conv_vert_adv_flux_SLT    (time, Z, YC, XC) float32 ...
    conv_vert_diff_flux_SLT   (time, Z, YC, XC) float32 ...
    lin_fs_correction_SLT     (time, YC, XC) float32 ...
    surface_flux_conv_SLT     (time, YC, XC) float32 ...
    total_tendency_SLT_truth  (time, Z, YC, XC) float64 ...
    UVEL                      (time, Z, YC, XG) float32 ...
    VVEL                      (time, Z, YG, XC) float32 ...
    WVEL                      (time, Zl, YC, XC) float32 ...
    WTHMASS                   (time, Zl, YC, XC) float32 ...
    WSLTMASS                  (time, Zl, YC, XC) float32 ...
    surForcS                  (time, YC, XC) float32 ...
    surForcT                  (time, YC, XC) float32 ...

In [10]:
ds = ds.chunk({'time':5})
ds.nbytes/1e9

52.322753328

In [ ]:
ds_tns.SALT.mean(dim=['time', 'Z']).drop('XC').plot()

In [ ]:
ds_no_polynya.SALT.mean(dim=['time', 'Z']).drop('XC').plot()

In [12]:
terms = ['conv_horiz_adv_flux_SLT', 'conv_horiz_diff_flux_SLT',
         'conv_vert_adv_flux_SLT', 'conv_vert_diff_flux_SLT',
         'surface_flux_conv_SLT', 'lin_fs_correction_SLT']

In [13]:
total = sum(ds[term] for term in terms)

In [15]:
total

<xarray.DataArray (time: 438, Z: 42, YC: 95, XC: 570)>
dask.array<add, shape=(438, 42, 95, 570), dtype=float32, chunksize=(5, 42, 95, 570), chunktype=numpy.ndarray>
Coordinates:
  * XC       (XC) float32 295.08334 295.25 295.4167 ... 29.75 29.916668
  * YC       (YC) float32 -77.87497 -77.7083 -77.54163 ... -62.374966 -62.2083
  * Z        (Z) float32 -5.0 -15.5 -27.0 -39.5 ... -5075.0 -5325.0 -5575.0
  * time     (time) datetime64[ns] 2005-01-06 2005-01-11 ... 2010-12-31

In [ ]:
total.isel(Z=0).mean(dim='time').plot()

In [ ]:
(ds/1e6).total_tendency_SLT_truth.isel(time=slice(0,73))

In [ ]:
(ds_no_polynya/1e6).total_tendency_SLT_truth.isel(time=slice(0,73))

In [ ]:
fig, (ax0,ax1) = plt.subplots(1, 2, figsize=(15,4))
(ds/1e6).total_tendency_TH_truth.isel(time=slice(0,73)).sum(dim=['XC', 'YC']).plot(y='Z', ax=ax0, 
                                              cbar_kwargs={'shrink':0.8, 'label':'Total Tendency [Sv]'})
ax0.set_title('tottend from full time ds - 2005')

(ds_no_polynya/1e6).total_tendency_TH_truth.isel(time=slice(0,73)).sum(dim=['XC', 'YC']).plot(y='Z', ax=ax1,
                                                         cbar_kwargs={'shrink':0.8, 'label':'Total Tendency [Sv]'},
                                                                                             vmax=15)
ax1.set_title('tottend from no polynya ds - 2006')

plt.tight_layout();